# LigthGBM con feature engineering

Para la entrega final y despues de haber hecho varias pruebas con diferentes modelos se decidio usar este modelo junto con una estrategia de feature engineering, a modo de garantizar una separabildad mayor para las variables respuesta medio-alto y medio-bajo y mayor numero de datos que amplien la interpretabilidad de los datos.

Este modelo se selecciono gracias a ser el que muestra mejor rendimiento

<h2>Importaciones importantes y reproducibilidad</h2>

In [ ]:
import os
import sys
import random
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler

import lightgbm as lgb
from lightgbm.callback import early_stopping, log_evaluation

# opcionales para explainability
import shap

RND = 42
os.environ['PYTHONHASHSEED'] = str(RND)
random.seed(RND)
np.random.seed(RND)

print("Python:", sys.version.splitlines()[0])
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("lightgbm:", lgb.__version__)
print("shap:", getattr(shap, '__version__', 'not installed'))


c:\Users\JUANJO\Documents\UdeA\saber_pro_predictor\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
pandas: 2.3.3
numpy: 2.3.3
lightgbm: 4.6.0
shap: 0.50.0


Esta funcion de pre procesamiento usa los mismos maps y parametros que los usados en el notebook "02 - prprocesado.ipynb", todos estos parametros se cargan desde un archivo .plk usando la libreria de joblib.

Por otra parte, tambien se asegura que se extraen los valores de la columna "ID" ya que será importante despues para la submision y se eliminaran las mismas columnas que se eliminaron en la anterior

In [24]:
def preprocess_new(df, params_path='data/preproc_params.pkl', one_hot_target=True, inplace=False):
    params = joblib.load(params_path)
    df = df if inplace else df.copy()

    for c in params.get('cols_to_drop', []):
        if c in df.columns:
            df.drop(columns=[c], inplace=True)

    if 'F_TIENEINTERNET.1' in df.columns:
        df.drop(columns=['F_TIENEINTERNET.1'], inplace=True)

    ordinal_maps = params.get('ordinal_mappings', {})
    for var, mapping in ordinal_maps.items():
        if var not in df.columns:
            continue
        df[var] = df[var].map(mapping)
        df[var] = df[var].where(pd.notna(df[var]), other=pd.NA)
        try:
            df[var] = df[var].astype('Int64')
        except Exception:
            df[var] = pd.to_numeric(df[var], errors='coerce').where(pd.notna(df[var]), pd.NA).astype('Int64')

    binary_maps = params.get('binary_maps', {})
    for col, mapping in binary_maps.items():
        if col in df.columns:
            df[col] = df[col].map(mapping)
            df[col] = df[col].where(pd.notna(df[col]), other=pd.NA).astype('Int64')

    enc_prg = params.get('E_PRGM_ACADEMICO_ENC_map', {})
    enc_dep = params.get('E_PRGM_DEPARTAMENTO_ENC_map', {})
    global_mean = params.get('global_mean', np.nan)

    if 'E_PRGM_ACADEMICO' in df.columns:
        df['E_PRGM_ACADEMICO_ENC'] = df['E_PRGM_ACADEMICO'].map(enc_prg)
        df['E_PRGM_ACADEMICO_ENC'].fillna(global_mean, inplace=True)
        df.drop(columns=['E_PRGM_ACADEMICO'], inplace=True)

    if 'E_PRGM_DEPARTAMENTO' in df.columns:
        df['E_PRGM_DEPARTAMENTO_ENC'] = df['E_PRGM_DEPARTAMENTO'].map(enc_dep)
        df['E_PRGM_DEPARTAMENTO_ENC'].fillna(global_mean, inplace=True)
        df.drop(columns=['E_PRGM_DEPARTAMENTO'], inplace=True)

    fill_values = params.get('fill_values', {})
    for col, info in fill_values.items():
        if col not in df.columns:
            continue
        strat = info.get('strategy', None)
        val = info.get('value', None)

        if strat in ('mean', 'mean_coerced', 'mean_rounded', 'mode', 'mode_fallback_non_numeric'):
            if pd.isna(val) or val is None:
                if strat.startswith('mean'):
                    fill_val = df[col].mean(skipna=True)
                else:
                    mode_series = df[col].mode(dropna=True)
                    fill_val = mode_series.iloc[0] if len(mode_series) > 0 else pd.NA
            else:
                fill_val = val
            df[col].fillna(fill_val, inplace=True)
            if strat == 'mean_rounded':
                try:
                    df[col] = df[col].round().astype('Int64')
                except Exception:
                    pass
        elif strat == 'mode' and val is None:
            mode_series = df[col].mode(dropna=True)
            if len(mode_series) > 0:
                df[col].fillna(mode_series.iloc[0], inplace=True)
        else:
            if not (pd.isna(val) or val is None):
                df[col].fillna(val, inplace=True)

    scale_params = params.get('scale_params', {})
    cols_to_scale = [c for c in params.get('cols_to_scale', []) if c in df.columns]
    for c in cols_to_scale:
        mn_mx = scale_params.get(c, None)
        if mn_mx is None:
            col_min = df[c].min(skipna=True)
            col_max = df[c].max(skipna=True)
        else:
            try:
                col_min, col_max = float(mn_mx[0]), float(mn_mx[1])
            except Exception:
                col_min = df[c].min(skipna=True)
                col_max = df[c].max(skipna=True)

        if pd.isna(col_min) or pd.isna(col_max) or col_max == col_min:
            continue
        df[c] = (df[c] - col_min) / (col_max - col_min)
        df[c] = df[c].clip(0.0, 1.0)

    ids = df["ID"]
    df.drop(columns=["ID"], inplace=True)
    X = df.copy()
    return X, ids

### 2. Se cargan los archivos necesarios para entrenar el modelo final

In [ ]:
preprocess_raw = False

y_df = pd.read_csv('data/y_preprocessed.csv')            # preferible: etiqueta (n,1)
y_onehot_df = pd.read_csv('data/y_one_hot_preprocessed.csv')  # one-hot (n,4)

# Convertir y a vector 1D; priorizar y_df si está bien formado
if y_df.shape[1] == 1:
    y_vec = y_df.iloc[:,0].values
else:
    # fallback: derivar desde one-hot
    y_vec = y_onehot_df.values.argmax(axis=1)

if preprocess_raw:
    X_raw = pd.read_csv('data/train.csv')
    X_proc, ids = preprocess_new(X_raw, params_path='data/preproc_params.pkl', inplace=False)
else:
    X_raw = pd.read_csv('data/X_preprocessed.csv')
    X_proc = X_raw.copy()
    
print("raw shapes:", X_raw.shape, y_df.shape, y_onehot_df.shape)
print("X_proc shape:", X_proc.shape, "y_vec shape:", y_vec.shape)

# Eliminar columnas que NO deben usarse
for drop_col in ['ID','F_PRIVADOLIBERTAD','F_INTERNET1']:
    if drop_col in X_proc.columns:
        X_proc.drop(columns=[drop_col], inplace=True)
        print("Dropped column:", drop_col)


raw shapes: (690861, 17) (690861, 1) (690861, 4)
X_proc shape: (690861, 17) y_vec shape: (690861,)


### 3. Se define la funcion de feature enginnering haciendo uso de las siguientes ideas:

- K-Fold target-mean encoding (OUT-OF-FOLD) para E_PRGM_ACADEMICO y E_PRGM_DEPARTAMENTO
- Estadísticas por grupo (mean/std/count) — por programa/dep sobre INDICADOR
- Frequency encoding para categorías de alta cardinalidad
- Binning (ordinalización) y flags para E_VALORMATRICULAUNIVERSIDAD y E_HORASSEMANATRABAJA
- Interacciones (producto / ratio) relevantes
- Missing indicator flags (la ausencia de información (no reportar matrícula, no reportar educación de padres) puede ser predictiva)
- Clustering sobre indicadores (k-means) → cluster id como feature
- Cross feature program x estrato
- Label encode program for potential NN embeddings

Todo esto resulta en un aumento de dimensiones hasta 41 columnas/variables diferentes.

In [ ]:
from sklearn.model_selection import KFold

def kfold_target_mean_series(X, y, col, n_splits=5, seed=RND):
    """Produce OOF target mean encoding as pd.Series aligned to X.index"""
    y_arr = np.asarray(y).ravel()
    out = pd.Series(index=X.index, dtype=float)
    gm = np.nanmean(y_arr.astype(float))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    for tr_idx, val_idx in kf.split(X):
        grp = pd.Series(y_arr[tr_idx]).groupby(X.iloc[tr_idx][col]).mean()
        out.iloc[val_idx] = X.iloc[val_idx][col].map(grp)
    out.fillna(gm, inplace=True)
    return out

def freq_encode_series(X, col):
    vc = X[col].value_counts()
    return X[col].map(vc).fillna(0)

def feature_engineer(X, y=None, do_kmeans=True, kmeans_k=8, save_maps_path='data/fe_maps.pkl'):
    """
    Aplica transformaciones y devuelve X_fe (DataFrame) y un dict 'saved' con maps/models.
    """
    X = X.copy()
    saved = {}

    # 1) Missing flags
    for c in X.columns:
        if X[c].isna().any():
            X[c + '_isna'] = X[c].isna().astype(int)

    # 2) Binning matricula y horas (ajustar bins si hace falta)
    if 'E_VALORMATRICULAUNIVERSIDAD' in X.columns:
        X['matricula_bin'] = pd.cut(X['E_VALORMATRICULAUNIVERSIDAD'].fillna(-1),
                                     bins=[-1,0,1,2,3,4,5,6,10], labels=False)
    if 'E_HORASSEMANATRABAJA' in X.columns:
        X['horas_bin'] = pd.cut(X['E_HORASSEMANATRABAJA'].fillna(-1),
                                 bins=[-1,0,1,2,3,4,10], labels=False)

    # 3) Interactions
    if {'E_HORASSEMANATRABAJA','INDICADOR_1'}.issubset(X.columns):
        X['hours_x_ind1'] = X['E_HORASSEMANATRABAJA'].fillna(0) * X['INDICADOR_1'].fillna(0)
    if {'E_VALORMATRICULAUNIVERSIDAD','F_ESTRATOVIVIENDA'}.issubset(X.columns):
        X['matric_x_estrato'] = X['E_VALORMATRICULAUNIVERSIDAD'].fillna(0) * X['F_ESTRATOVIVIENDA'].fillna(0)

    # 4) Frequency encoding program & department
    if 'E_PRGM_ACADEMICO_ENC' in X.columns:
        prg_freq = X['E_PRGM_ACADEMICO_ENC'].value_counts()
        X['prg_freq'] = X['E_PRGM_ACADEMICO_ENC'].map(prg_freq).fillna(0)
        saved['prg_freq_map'] = prg_freq.to_dict()
    if 'E_PRGM_DEPARTAMENTO_ENC' in X.columns:
        dep_freq = X['E_PRGM_DEPARTAMENTO_ENC'].value_counts()
        X['dep_freq'] = X['E_PRGM_DEPARTAMENTO_ENC'].map(dep_freq).fillna(0)
        saved['dep_freq_map'] = dep_freq.to_dict()

    # 5) KFold target-mean enc (OOF) for program/dep (requires y)
    if y is not None:
        try:
            if 'E_PRGM_ACADEMICO_ENC' in X.columns:
                X['prg_tgt_mean'] = kfold_target_mean_series(X, y, 'E_PRGM_ACADEMICO_ENC', n_splits=5, seed=RND)
            if 'E_PRGM_DEPARTAMENTO_ENC' in X.columns:
                X['dep_tgt_mean'] = kfold_target_mean_series(X, y, 'E_PRGM_DEPARTAMENTO_ENC', n_splits=5, seed=RND)
        except Exception as e:
            print("Warning: target-mean encoding failed:", e)

    # 6) Aggregates by program (indicator means/std/count)
    agg_cols = [c for c in ['INDICADOR_1','INDICADOR_2','INDICADOR_3','INDICADOR_4'] if c in X.columns]
    if 'E_PRGM_ACADEMICO_ENC' in X.columns and len(agg_cols)>0:
        agg = X.groupby('E_PRGM_ACADEMICO_ENC')[agg_cols].agg(['mean','std','count'])
        agg.columns = ['_'.join(col).strip() for col in agg.columns.values]
        # merge (map)
        for col in agg.columns:
            X[f'prg_{col}'] = X['E_PRGM_ACADEMICO_ENC'].map(agg[col]).fillna(0)
        saved['prg_agg_map'] = agg

    # 7) KMeans clustering on indicators
    if do_kmeans and len(agg_cols) >= 2:
        km = KMeans(n_clusters=kmeans_k, random_state=RND)
        X['cluster_indic'] = km.fit_predict(X[agg_cols].fillna(0))
        saved['kmeans_model'] = km

    # 8) Cross feature program x estrato
    if {'E_PRGM_ACADEMICO_ENC','F_ESTRATOVIVIENDA'}.issubset(X.columns):
        X['prg_estrato'] = X['E_PRGM_ACADEMICO_ENC'].astype(str) + '_' + X['F_ESTRATOVIVIENDA'].astype(str)
        prg_estrato_freq = X['prg_estrato'].value_counts()
        X['prg_estrato_freq'] = X['prg_estrato'].map(prg_estrato_freq).fillna(0)
        saved['prg_estrato_freq_map'] = prg_estrato_freq.to_dict()

    # 9) Label encode program for potential NN embeddings
    if 'E_PRGM_ACADEMICO_ENC' in X.columns:
        le = LabelEncoder()
        X['prg_idx'] = le.fit_transform(X['E_PRGM_ACADEMICO_ENC'].astype(str).fillna('NA'))
        saved['prg_label_encoder'] = le

    # Save maps
    if save_maps_path:
        os.makedirs(os.path.dirname(save_maps_path), exist_ok=True)
        joblib.dump(saved, save_maps_path)

    return X, saved


Se hace un mapeo de cada una de las variables hacia un valor numerico ordinal, de modo que el modelo logre comprender la estructura de las labels en una estructura de dato numerica ordinal (no funciona con one hot encoding)

In [ ]:
# Mapeo ordinal → numérico
label_map = {
    "bajo": 0,
    "medio-bajo": 1,
    "medio-alto": 2,
    "alto": 3
}

y_numeric = y_df["RENDIMIENTO_GLOBAL"].map(label_map).astype(int)
print("Ejemplo conversión de etiquetas:")
print(pd.DataFrame({"original": y_vec, "numeric": y_numeric}).head())

# Aplicar feature engineering
X_fe, fe_maps = feature_engineer(
    X_proc,
    y=y_numeric,
    do_kmeans=True,
    kmeans_k=8,
    save_maps_path="data/fe_maps.pkl"
)

print("X_fe shape:", X_fe.shape)

Ejemplo conversión de etiquetas:
     original  numeric
0  medio-alto        2
1        bajo        0
2        bajo        0
3        alto        3
4  medio-bajo        1
X_fe shape: (690861, 41)


### Eliminamos las columnas que no queramos usar, separamos entre entrenamiento, test y validacion y nos aseguramos de que todas las estructuras de datos tengan el formato que especificamente queremos

In [7]:
# Convert all object columns to category → integer codes
for col in X_proc.select_dtypes(include=["object"]).columns:
    X_proc[col] = X_proc[col].astype("category").cat.codes

In [9]:
# Cell 6 — split (estratificado)
from sklearn.model_selection import train_test_split

# Excluir columnas que no queremos entrenar (por seguridad)
for c in ['ID','F_PRIVADOLIBERTAD','F_INTERNET1']:
    if c in X_fe.columns:
        X_fe.drop(columns=[c], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X_fe, y_numeric, test_size=0.10, random_state=RND, stratify=y_numeric)
print("Train:", X_train.shape, y_train.shape, "Test:", X_test.shape, y_test.shape)

Train: (621774, 41) (621774,) Test: (69087, 41) (69087,)


In [10]:
for col in X_train.select_dtypes(include=["object"]).columns:
    X_train[col] = X_train[col].astype("category").cat.codes

for col in X_test.select_dtypes(include=["object"]).columns:
    X_test[col] = X_test[col].astype("category").cat.codes

### 4. Celda 16 — Entrenamiento CV con StratifiedKFold y LightGBM
- Objetivo: hacer validación cruzada estratificada (5 folds) sobre X_train/y_train para evaluar estabilidad del modelo y obtener métricas por fold.
- Preparación:
    - Se resetean índices de X_train y y_train para evitar problemas al indexar.
    - Se crea un `StratifiedKFold(n_splits=5, shuffle=True, random_state=RND)` para mantener la proporción de clases en cada fold.
- Bucle por fold:
    - Se obtienen índices de entrenamiento/validación y se seleccionan con `iloc` (evita KeyError por índices no coincidentes).
    - Se instancia un `lgb.LGBMClassifier` con hiperparámetros fijos (multiclass, learning_rate 0.03, num_leaves 64, regularización, etc.).
    - Se entrena con `fit(..., eval_set=[(X_val,y_val)], eval_metric="multi_logloss", callbacks=[log_evaluation(period=100), early_stopping(80)])` para usar evaluación en validación y early stopping.
    - Se predice sobre la partición de validación y se calculan accuracy, macro-F1 y la matriz de confusión. Se acumulan métricas y modelos.
- Resultados:
    - Lista `accs` y `f1s` por fold, `cm_sum` con la suma agregada de las matrices de confusión, `models` contiene un modelo por fold.
    - Se imprime resumen de resultados (media y desviación).

In [ ]:
from sklearn.model_selection import StratifiedKFold
from lightgbm.callback import early_stopping, log_evaluation
import time
import math

# IMPORTANT: reset indexes to avoid KeyError
X_train = X_train.reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RND)
fold = 0
accs, f1s = [], []
cm_sum = None
models = []

for train_idx, val_idx in skf.split(X_train, y_train):
    fold += 1
    
    # FIX: use iloc to index properly
    X_tr = X_train.iloc[train_idx]
    y_tr = y_train.iloc[train_idx]
    X_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]

    model = lgb.LGBMClassifier(
        objective="multiclass",
        num_class=len(np.unique(y_train)),
        boosting_type="gbdt",
        learning_rate=0.03,
        n_estimators=3000,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=RND,
        verbose=-1
    )

    print(f"\n--- Fold {fold} training ---")
    t0 = time.time()
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="multi_logloss",
        callbacks=[log_evaluation(period=100), early_stopping(stopping_rounds=80)]
    )
    t1 = time.time()
    print(f"Fold {fold} trained in {t1-t0:.1f}s; best_iter={model.best_iteration_}")

    y_val_pred = model.predict(X_val, num_iteration=model.best_iteration_)
    acc = accuracy_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred, average='macro')
    accs.append(acc); f1s.append(f1)
    print(f"Fold {fold} Acc: {acc:.4f} Macro-F1: {f1:.4f}")

    cm = confusion_matrix(y_val, y_val_pred)
    cm_sum = cm if cm_sum is None else cm_sum + cm

    models.append(model)

print("\n=== CV Results ===")
print("Accuracy per fold:", accs)
print("Macro-F1 per fold:", f1s)
print("Mean Acc: %.4f (std %.4f)" % (np.mean(accs), np.std(accs)))
print("Mean Macro-F1: %.4f (std %.4f)" % (np.mean(f1s), np.std(f1s)))
print("Aggregated confusion matrix:\n", cm_sum)



--- Fold 1 training ---
Training until validation scores don't improve for 80 rounds
[100]	valid_0's multi_logloss: 1.20187
[200]	valid_0's multi_logloss: 1.1904
[300]	valid_0's multi_logloss: 1.18732
[400]	valid_0's multi_logloss: 1.18627
[500]	valid_0's multi_logloss: 1.18563
[600]	valid_0's multi_logloss: 1.18522
[700]	valid_0's multi_logloss: 1.18497
[800]	valid_0's multi_logloss: 1.18483
[900]	valid_0's multi_logloss: 1.18481
Early stopping, best iteration is:
[846]	valid_0's multi_logloss: 1.18477
Fold 1 trained in 180.5s; best_iter=846
Fold 1 Acc: 0.4422 Macro-F1: 0.4308

--- Fold 2 training ---
Training until validation scores don't improve for 80 rounds
[100]	valid_0's multi_logloss: 1.2012
[200]	valid_0's multi_logloss: 1.19015
[300]	valid_0's multi_logloss: 1.18683
[400]	valid_0's multi_logloss: 1.18588
[500]	valid_0's multi_logloss: 1.18551
[600]	valid_0's multi_logloss: 1.18535
[700]	valid_0's multi_logloss: 1.18518
[800]	valid_0's multi_logloss: 1.18516
Early stopping, b


### 5. Entrenamiento final y evaluación sobre test
- Objetivo: entrenar un modelo final sobre todo el conjunto de entrenamiento (X_train completo) y evaluar su rendimiento en X_test.
- Proceso:
    - Se crea otro `LGBMClassifier` con la misma configuración.
    - Se llama a `fit` usando `eval_set=[(X_test,y_test)]` y los mismos callbacks (log_evaluation + early_stopping) para controlar iteraciones y ver rendimiento en el test durante el entrenamiento.
    - Se recupera `final_model.best_iteration_` y se predice con `predict(..., num_iteration=best_iteration_)`.
- Salidas impresas:
    - Best iteration, accuracy y macro-F1 sobre X_test.
    - `classification_report` con precision/recall/f1 por clase y la `confusion_matrix`.
- Nota práctica: este modelo final es el que luego se usa para predecir sobre datos nuevos (test set de producción).

In [11]:
# Cell 8 — train final on X_train (all) and evaluate on X_test
final_model = lgb.LGBMClassifier(
    objective="multiclass",
    num_class=len(np.unique(y_numeric)),
    boosting_type="gbdt",
    learning_rate=0.03,
    n_estimators=3000,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=RND,
    verbose=-1
)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric="multi_logloss",
    callbacks=[log_evaluation(period=100), early_stopping(stopping_rounds=80)]
)

print("Final best_iter:", final_model.best_iteration_)
y_test_pred = final_model.predict(X_test, num_iteration=final_model.best_iteration_)
print("Test accuracy:", accuracy_score(y_test, y_test_pred))
print("Test macro-F1:", f1_score(y_test, y_test_pred, average='macro'))
print("Classification report:\n", classification_report(y_test, y_test_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_test_pred))

Training until validation scores don't improve for 80 rounds
[100]	valid_0's multi_logloss: 1.2206
[200]	valid_0's multi_logloss: 1.20789
[300]	valid_0's multi_logloss: 1.20421
[400]	valid_0's multi_logloss: 1.2034
[500]	valid_0's multi_logloss: 1.2027
[600]	valid_0's multi_logloss: 1.20231
[700]	valid_0's multi_logloss: 1.20215
[800]	valid_0's multi_logloss: 1.20195
Early stopping, best iteration is:
[819]	valid_0's multi_logloss: 1.20193
Final best_iter: 819
Test accuracy: 0.43216524092810515
Test macro-F1: 0.42269458837501206
Classification report:
               precision    recall  f1-score   support

           0     0.4401    0.6315    0.5187     17256
           1     0.3199    0.2821    0.2998     17186
           2     0.3361    0.2630    0.2951     17123
           3     0.6093    0.5483    0.5772     17522

    accuracy                         0.4322     69087
   macro avg     0.4263    0.4312    0.4227     69087
weighted avg     0.4273    0.4322    0.4237     69087

Confus

### 6. Interpretacion de los resultados

- Clase 0 → mucho recall (0.63): “casos claramente malos”
- Clase 3 → buen recall (0.55): “casos claramente buenos”
- Clase 1 y 2 → casi aleatorias porque están “entre medios”

Esto ocurre porque el espacio de features no tiene fronteras claras para separar las clases intermedias.
El modelo NO tiene información suficiente para distinguir finamente esos grupos.
Si las variables no contienen información predictiva
O el fenómeno es inherentemente difuso
O el problema es ordinal, pero se trata como “clases duras”

Aun y con esto, la mejora respecto a los demas en kaggle submission es sustancial y esto lo vuelve candidato a seleccion por motivos de tiempo y trabajo.

In [39]:
y_test_pred[:10] , type(y_test_pred) # mostrar primeras 10 predicciones

(array([2, 0, 1, 0, 2, 0, 1, 1, 1, 1]), numpy.ndarray)

### 7. Creacion del Submission a Kaggle

- Definición de utilidad
    - numeric_to_label: función que convierte arreglos numéricos {0,1,2,3} a etiquetas de texto ("bajo", "medio-bajo", "medio-alto", "alto") usando `np.vectorize`.

- Carga y vista rápida del set de test
    - test_df: se carga `data/test.csv` y se inspecciona con `head()`.

- Preprocesado y extracción de IDs
    - X_test_new, ids_test_new = preprocess_new(test_df): aplica el mismo preprocesado usado en entrenamiento (mapa de ordinales/binning, imputaciones, escalado, y extracción de la columna `ID`). `ids_test_new` se guarda para la salida final.

- Feature engineering sobre los datos nuevos
    - X_test_new_fe = feature_engineer(X_test_new, ...): genera nuevas variables (flags de missing, bins, interacciones, frequency encodings, clustering, label-encoding, etc.). Nota: aquí no se pasa `y`, por lo que no se hace target-mean OOF; además el KMeans y LabelEncoder se ajustan de nuevo sobre el test (puede producir diferencias frente a lo usado en entrenamiento si no se reutilizan los mapas/ modelos guardados).

- Limpieza puntual
    - Se elimina `prg_estrato`, probablemente porque no coincide con las columnas usadas por el modelo final.

- Predicción y comprobaciones
    - y_test_proba = final_model.predict(...): se obtienen las predicciones numéricas del modelo final. Atención: el nombre `y_test_proba` es engañoso — aquí hay etiquetas predichas, no probabilidades.
    - Se muestran ejemplos y se verifican shapes (celdas 31, 33).

- Conversión a etiquetas textuales y creación del CSV de submission
    - rendimiento_global_pred = numeric_to_label(y_test_proba): mapea los números a las etiquetas finales.
    - kaggle_submission: DataFrame con `ID` y `RENDIMIENTO_GLOBAL`.
    - kaggle_submission.to_csv(...): escribe `kaggle_submission_2.csv` listo para subir.


In [14]:
def numeric_to_label(arr):
    mapping = {
        0: "bajo",
        1: "medio-bajo",
        2: "medio-alto",
        3: "alto"
    }
    
    # Vectorizamos el diccionario para aplicarlo a cada elemento
    vectorized_map = np.vectorize(lambda x: mapping.get(x, "valor-desconocido"))
    return vectorized_map(arr)

In [26]:
test_df = pd.read_csv('data/test.csv')

In [27]:
test_df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [29]:
X_test_new, ids_test_new = preprocess_new(test_df)

C:\Users\JUANJO\AppData\Local\Temp\ipykernel_3948\2178492099.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['E_PRGM_ACADEMICO_ENC'].fillna(global_mean, inplace=True)
C:\Users\JUANJO\AppData\Local\Temp\ipykernel_3948\2178492099.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [30]:
X_test_new.head()

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_ENC,E_PRGM_DEPARTAMENTO_ENC
0,0.000000,1,1,3,0,6,0,1,1,0,2,0.499239,0.449692,0.990625,0.743976,0.276905,0.584647
1,0.666667,4,3,2,0,4,0,1,0,0,6,0.345510,0.581109,0.925000,0.975904,0.347838,0.831733
2,0.966667,3,0,3,0,3,0,1,0,0,4,0.433790,0.468172,0.918750,0.743976,0.726086,0.761121
3,0.400000,3,3,1,1,1,0,1,0,1,1,0.243531,0.837782,0.678125,0.885542,0.353210,0.387345
4,0.966667,4,2,2,0,4,0,1,0,0,4,0.318113,0.581109,0.956250,0.861446,0.396928,0.695386


In [35]:
X_test_new_fe, _ = feature_engineer(
    X_test_new,
    do_kmeans=True,
    kmeans_k=8,
    save_maps_path="data/fe_maps.pkl"
)             

In [48]:
X_test_new_fe.head()

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,prg_INDICADOR_3_mean,prg_INDICADOR_3_std,prg_INDICADOR_3_count,prg_INDICADOR_4_mean,prg_INDICADOR_4_std,prg_INDICADOR_4_count,cluster_indic,prg_estrato,prg_estrato_freq,prg_idx
0,0.000000,1,1,3,0,6,0,1,1,0,...,0.795567,0.196588,5286,0.826230,0.203492,5286,4,0.27690477849910805_3,1513,95
1,0.666667,4,3,2,0,4,0,1,0,0,...,0.832837,0.193746,195,0.749382,0.210195,195,5,0.3478380318863289_2,53,169
2,0.966667,3,0,3,0,3,0,1,0,0,...,0.837931,0.178425,964,0.791106,0.190470,964,6,0.7260863431985197_3,412,716
3,0.400000,3,3,1,1,1,0,1,0,1,...,0.822151,0.179429,16861,0.828942,0.184349,16861,3,0.3532100667724726_1,3665,176
4,0.966667,4,2,2,0,4,0,1,0,0,...,0.819981,0.182322,22298,0.783877,0.207804,22298,1,0.39692784238156564_2,7654,238


In [53]:
X_test_new_fe.drop(columns=["prg_estrato"], inplace=True)

In [ ]:
y_test_proba = final_model.predict(X_test_new_fe, num_iteration=final_model.best_iteration_)

In [56]:
y_test_proba[:10]

array([1, 1, 1, 0, 1, 1, 1, 3, 1, 1])

In [58]:
ids_test_new.head()

0    550236
1     98545
2    499179
3    782980
4    785185
Name: ID, dtype: int64

In [57]:
ids_test_new.shape, y_test_proba.shape

((296786,), (296786,))

In [61]:
rendimiento_global_pred = numeric_to_label(y_test_proba)

In [62]:
kaggle_submission = pd.DataFrame({
    "ID": ids_test_new,
    "RENDIMIENTO_GLOBAL": rendimiento_global_pred
})

In [63]:
kaggle_submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-bajo
3,782980,bajo
4,785185,medio-bajo


In [66]:
kaggle_submission.to_csv('kaggle_submission_2.csv', index=False)

### Conclusión

- El pipeline (preprocesado + feature engineering + LightGBM) aporta buena separación para clases extremas (bajo/alto) pero falla en distinguir las clases intermedias (medio-bajo/medio-alto).  
- Métricas CV y test muestran estabilidad en extremos pero baja discriminación en el centro, indicando información insuficiente o ruido en las features para ese nivel de granularidad.  
- **Recomendaciones prácticas:** considerar modelos/objetivos ordinales, recopilar features más informativas, y reutilizar mapas/modelos de FE (encoders, KMeans) al predecir para mejorar consistencia en producción.